# Make Text Searchable

In [ ]:
# Import python packages
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

# Write directly to the app
st.markdown("### Use Vector Embeddings to make data searchable and only send the relevant data to the LLM for quick answers")

# Get the current credentials
session = get_active_session()



letters = session.table('BUILD_UK.DATA.LETTERS_TO_MP')

SMEDIA = session.table('BUILD_UK.DATA.V_SOCIAL_MEDIA')

INCIDENTS = session.table('BUILD_UK.DATA.INCIDENTS').join_table_function('flatten','GENERATED_EVENTS').select(F.col('VALUE').alias('INCIDENT'))
INCIDENTS = INCIDENTS.join_table_function('flatten','INCIDENT').select('VALUE')

st.markdown('#### Here are the letters')
letters

st.markdown('#### Here is the social media')
SMEDIA

st.markdown('#### Here are the Incidents')
INCIDENTS


object = letters.select(F.col('LETTER').alias('OBJECT'))\
.union(SMEDIA.select(F.col('V').astype(StringType()).alias('OBJECT')))



#### Present both letters and social media comments as an object

In [ ]:
object = letters.select(F.col('LETTER').alias('OBJECT'))\
.union(SMEDIA.select(F.col('V').astype(StringType()).alias('OBJECT')))\
.union(INCIDENTS.select(F.col('VALUE').astype(StringType()).alias('OBJECT')))

object.write.mode('overwrite').save_as_table("DATA.EVENT_TEXT")
st.write(object)

#### Use Vector Embeddings to put the unstructured data into an array using built in text embeddings

![alt text](https://docs.snowflake.com/en/_images/vector-similarity-vectors-example.png "Title")

In [ ]:
embeds = object.with_column('EMBED',F.call_function('SNOWFLAKE.CORTEX.EMBED_TEXT_768',
                                                    F.lit('snowflake-arctic-embed-m'),
                                                    F.col('OBJECT'))).cache_result()

In [ ]:
embeds

#### Do something with the data

In [ ]:
question = st.text_input('Type in your question:','Produce a News report about Food poisoning')

In [ ]:
poison_search = embeds.with_column('QUESTION',F.lit(question))



#### Embed the Question the same way as the data

In [ ]:
poison_search = poison_search.with_column('EMBEDQ',F.call_function('SNOWFLAKE.CORTEX.EMBED_TEXT_768',
                                                    F.lit('snowflake-arctic-embed-m'),
                                                    F.col('QUESTION'))).cache_result()

#### Use the Vector Cosine Similarity function to rank each row based on how close the vectors are - the closer the data is to the question, the higher the ranking

In [ ]:
poison_similar = poison_search.with_column('search',F.call_function('VECTOR_COSINE_SIMILARITY'
                                           ,F.col('EMBED'),
                                          F.col('EMBEDQ')))

poison_similar.sort(F.col('SEARCH').desc()) 

#### Sort by relavance and only show the top 5

In [ ]:
poison_similar = poison_similar.sort(F.col('SEARCH').desc()).limit(5).cache_result()

#### Tidy up the dataframe to view only the question and the relevant datasets

In [ ]:
poison_similar.select('OBJECT','QUESTION')

### Use Cortex Complete in the same way as before to provide a readable answer

In [ ]:
### link to the different llm functions and what region supports them - https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions

poison_relevent = poison_similar.select(F.array_agg('OBJECT').alias('OBJECT'))
ANSWER = poison_relevent.with_column('ANSWER',
                                    F.call_function('SNOWFLAKE.CORTEX.COMPLETE',F.lit('mixtral-8x7b'),
                                                   F.concat(F.lit(question),
                                                           F.lit(' Based on the following data: '),
                                                           F.col('OBJECT').astype(StringType()),
                                                           F.lit('Only provide the answer in markdown format '),
                                                           F.lit('Do not provide additional commentary'))))

In [ ]:
ANSWER

In [ ]:
st.write(ANSWER.select('ANSWER').to_pandas().ANSWER.iloc[0])

### Create a Search Service
You can see how you can make text searchable.  Another way is to create a search service.  this embeds the text as a background process and also dynamically refreshes the service as new data arrives.  There is also a tool within the cortex studio (home page) which creates the same SQL within the UI.

In [ ]:
CREATE OR REPLACE  CORTEX SEARCH SERVICE DATA.NORTHERN_ACTIVITY
  ON OBJECT
  WAREHOUSE = BUILD_UK_WAREHOUSE
  TARGET_LAG = '1 hour'
  COMMENT = 'Searching through all event text'
  AS SELECT * FROM DATA.EVENT_TEXT;

In [ ]:
DESCRIBE CORTEX SEARCH SERVICE DATA.NORTHERN_ACTIVITY

### Test out the Search Service with Streamlit

In [ ]:
# Import python packages
import streamlit as st
from snowflake.core import Root
from snowflake.snowpark.context import get_active_session

# Constants
DB = "BUILD_UK"
SCHEMA = "DATA"
SERVICE = "NORTHERN_ACTIVITY"
BASE_TABLE = "BUILD_UK.DATA.EVENT_TEXT"


def get_column_specification():
    """
    Returns the name of the search column and a list of the names of the attribute columns
    for the provided cortex search service
    """
    session = get_active_session()
    search_service_result = session.sql(f"DESC CORTEX SEARCH SERVICE {DB}.{SCHEMA}.{SERVICE}").collect()[0]
    #st.session_state.attribute_columns = search_service_result.attribute_columns.split(",")
    st.session_state.search_column = search_service_result.search_column
    st.session_state.columns = search_service_result.columns.split(",")

def init_layout():
    st.title("Cortex AI Search")
    st.markdown(f"Querying service: `{DB}.{SCHEMA}.{SERVICE}`".replace('"', ''))

def query_cortex_search_service(query, filter={}):
    """
    Queries the cortex search service in the session state and returns a list of results
    """
    session = get_active_session()
    cortex_search_service = (
        Root(session)
        .databases[DB]
        .schemas[SCHEMA]
        .cortex_search_services[SERVICE]
    )
    context_documents = cortex_search_service.search(
        query,
        columns=st.session_state.columns,
        filter=filter,
        limit=st.session_state.limit)
    return context_documents.results

@st.cache_data
def distinct_values_for_attribute(col_name, is_array_attribute=False):
    session = get_active_session()
    if is_array_attribute:
        values = session.sql(f'''
        SELECT DISTINCT value FROM {BASE_TABLE},
        LATERAL FLATTEN(input => {col_name})
        ''').collect()
    else:
        values = session.sql(f"SELECT DISTINCT {col_name} AS VALUE FROM {BASE_TABLE}").collect()
    return [ x["VALUE"].replace('"', "") for x in values ]

def init_search_input():
    st.session_state.query = st.text_input("Query")

def init_limit_input():
    st.session_state.limit = st.number_input("Limit", min_value=1, value=5)



def display_search_results(results):
    """
    Display the search results in the UI
    """
    st.subheader("Search results")
    for i, result in enumerate(results):
        result = dict(result)
        container = st.expander(f"[Result {i+1}]", expanded=True)

        # Add the result text.
        container.markdown(result[st.session_state.search_column])

        # Add the attributes.
        for column, column_value in sorted(result.items()):
            if column == st.session_state.search_column:
                continue
            container.markdown(f"**{column}**: {column_value}")



def main():
    init_layout()
    get_column_specification()
    init_limit_input()
    init_search_input()

    if not st.session_state.query:
        return
    results = query_cortex_search_service(
        st.session_state.query
    )
    display_search_results(results)


if __name__ == "__main__":
    #st.set_page_config(page_title="Cortex AI Search and Summary", layout="wide")
    main()